In [1]:
from tasks import crear_factura_proveedores
from rich.console import Console
import requests

In [2]:
df = crear_factura_proveedores.read_data()

In [2]:
session_id = crear_factura_proveedores.login(
    user_name='manager',
    password='1234',
    company_db='SBO_TDPC_TEST'
)



In [4]:
URL_BASE = crear_factura_proveedores.URL_BASE

url = f"{URL_BASE}/b1s/v1/BusinessPartners('P20610242627')"

headers = {
    'Content-Type': 'application/json',
    'Cookie': f'B1SESSION={session_id}; ROUTEID=.node1'
}

re

Buscando Código DET D027

In [13]:
main_window[6]

IndexError: list index out of range